In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../input/fake-news/train.csv')

In [ ]:
df.head

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label', axis=1)

In [ ]:
y = df['label']

In [ ]:
y.value_counts()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dropout

In [ ]:
voc_size = 5000

#### One Hot Representation

In [ ]:
messages = X.copy()

In [ ]:
messages['title'][1]

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    news = re.sub('a-zA-Z', ' ', messages['title'][i])
    news = news.lower()
    news = news.split()
    news = [ps.stem(word) for word in news if not word in stopwords.words('english')]
    news = ' '.join(news)
    corpus.append(news)

In [ ]:
corpus

In [ ]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

#### Embedding Representation

In [ ]:
sen_len = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sen_len)
print(embedded_docs)

In [ ]:
embedded_docs[0]

In [ ]:
# Model creation
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sen_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
# Model creation (Bidirectional LSTM)
embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sen_len))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

In [ ]:
len(embedded_docs), y.shape

In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape, y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state=42)

#### Model Training

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [ ]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [ ]:
y_pred=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)